# Assignment 10: X-Ray Pneumonia Detection

In this assignment, you are tasked with developing your own classifier for pneumonia in X-ray images. You will go through to the complete ML development cycle from loading and preprocessing your data to evaluating your models.

Download and extract the X-Ray dataset from Ilias to the same directory as your Jupyter notebook. The data is already split into a training, validation, and testing set. The dataset originates from the paper [Identifying Medical Diagnoses and Treatable Diseases by Image-Based Deep Learning
](https://www.cell.com/cell/fulltext/S0092-8674(18)30154-5).

You may use any packages, we encountered during the exercises (numpy, matplotlib, scikit-learn, scikit-image, pandas, pytorch) as well as the Python standard library.

You should (at least) address the following points in your development process:

- The dataset is imbalanced. Do at least one of the following:
    - Augment your dataset by including rotated, flipped, or brightened images. This will also improve the generalization capabilities of your model.
    - or: Modify your objective function by weighting the classes differently.
- Optimize the hyperparameters of your models using grid-search or random-search on the validation set.
- Consider at least two classes of models, e.g. CNN and SVM. At least one of your model classes should be some type of neural network implemented in PyTorch.
- After the hyperparameter optimization, select the best-performing models of each class. Evaluate these models on the testing data and visualize your results.


*Note*: You have 2 weeks to complete this assignment. The assignment is optional for B.Sc.Data Science students, who have an alternative easier assignment 10 on clustering. If you already have successfully submitted 7 assignments, you do not need to submit this assignment.
Presenting this assignment in a video counts as **2 video presentations**. Please indicate if you wish to present at the top of the notebook.

- *Put your names here*

**Indicate here whether you want to present.**

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import skimage
import skimage.io
import skimage.transform
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import sklearn
import os
import torchvision
from tqdm import tqdm
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report



In [2]:
# make data loader in this cell
# print(os.listdir(os.path.join('chest_xray/train','NORMAL')))
def load_data(directory):
    print("load images:")
    label=[]
    normal_images=[]
    pneumonia_images=[]
    # subFolder= ['NORMAL','PNEUMONIA']
    # for d in subFolder:
    directories=[d for d in os.listdir(os.path.join(directory,'NORMAL'))]
    for f in tqdm(directories,position=0):
        I=skimage.io.imread(os.path.join(os.path.join(directory,'NORMAL'),f))
        if len(I.shape) ==2:
            I=skimage.color.gray2rgb(I)
        normal_images.append(skimage.transform.resize(I,(128,128)))
        label.append(0)
    directories=[d for d in os.listdir(os.path.join(directory,'PNEUMONIA'))]
    for f in tqdm(directories,position=0):
        I=skimage.io.imread(os.path.join(os.path.join(directory,'PNEUMONIA'),f))
        if len(I.shape)==2:
            I=skimage.color.gray2rgb(I)
        pneumonia_images.append(skimage.transform.resize(I,(128,128)))
        label.append(1)

    
    anglelist=[90,270,180]

    if len(normal_images)<len(pneumonia_images):
        image_num_diff=len(pneumonia_images)-len(normal_images)
        for i in tqdm(range(image_num_diff)):
            idx=random.randint(0,len(normal_images)-1)
            angle_idx=random.randint(0,2)
            # angle=random.randint(-90,90)
            normal_images.append(skimage.transform.rotate(normal_images[idx],anglelist[angle_idx] ))
            label.append(0)
    elif len(normal_images)>len(pneumonia_images):
        image_num_diff=-len(pneumonia_images)+len(normal_images)
        for i in range(image_num_diff):
            idx=random.randint(0,len(pneumonia_images-1))
            angle_idx=random.randint(0,2)
            # angle=random.randint(-90,90)
            pneumonia_images.append(skimage.transform.rotate(pneumonia_images[idx],anglelist[angle_idx]) )
            label.append(1)
    
    images=normal_images+pneumonia_images
    images,label=np.array(images),np.array(label)
    return images,label


X_train,y_train= load_data('chest_xray/train')
X_val,y_val,=load_data('chest_xray/val')
X_test,y_test=load_data('chest_xray/test')


    
            

100%|██████████| 156/156 [00:00<00:00, 460.18it/s]


In [10]:
# test the dataset and define dateloader
device=torch.device('cuda')
print(X_train[0].shape)
class BasicDataset(object):
    def __init__(self, X, y,transform):
        self.X = np.moveaxis(X,-1,1)
        self.y = y
        self.transform=transform
    
    def __getitem__(self, idx):
        return dict(X=self.transform(torch.Tensor(self.X[idx])).numpy(), y=self.y[idx])
    
    def __len__(self):
        return self.X.shape[0]


(128, 128, 3)


In [17]:
# define the nn by pytorch
class RNNClassifer(nn.Module):
    def __init__(self,hidden_size):
        super(RNNClassifer,self).__init__()
        self.cnnLayer1=nn.Sequential(
            nn.Conv2d(3,16,3,2,padding=2),
            nn.ReLU(),
            nn.Conv2d(16,16,3,2),
            nn.ReLU()
        )

        self.cnnLayer2=nn.Sequential(
            nn.Conv2d(16,32,3,1,padding=2),
            nn.ReLU(),
            nn.Conv2d(32,32,3,1),
            nn.ReLU()
        )

        self.cnnLayer3=nn.Sequential(
            nn.Conv2d(32,48,3,1,padding=2),
            nn.ReLU(),
            nn.Conv2d(48,48,3,1),
            nn.ReLU()
        )

        self.shortcut_cnn1=nn.Sequential(
            nn.Conv2d(3,16,1,4),
            nn.ReLU()
        )
        self.shortcut_cnn2=nn.Sequential(
            nn.Conv2d(16,32,1,1),
            nn.ReLU()
        )
        self.shortcut_cnn3=nn.Sequential(
            nn.Conv2d(32,48,1,1),
            nn.ReLU()
        )

        self.poollayer=nn.MaxPool2d(3,2)
        self.gaplayeer=nn.AvgPool2d(3,2)
        self.dense_layer=nn.Sequential(
            nn.Linear(48*1*1,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.Dropout(0.3),
            nn.ReLU()
        )
        self.output_layer=nn.Linear(hidden_size,2)
    def forward(self,X):
        cnn_out=self.cnnLayer1(X)
        shortcut_out=self.shortcut_cnn1(X)

        out=self.poollayer(cnn_out+shortcut_out)

        cnn_out=self.cnnLayer2(out)
        shortcut_out=self.shortcut_cnn2(out)
        out=self.poollayer(cnn_out+shortcut_out)

        cnn_out=self.cnnLayer3(out)
        shortcut_out=self.shortcut_cnn3(out)
        out=self.poollayer(cnn_out+shortcut_out)

        out=self.gaplayeer(out)

        # print(out.shape)
        out=out.view(-1,48*1*1)
        out=self.dense_layer(out)
        out=self.output_layer(out)
        return out
        

def train_model(model,dataset,learing_rate,batch_size,epochs):
    optimizer=optim.Adam(model.parameters(),lr=learing_rate)
    loss_list=[]
    acc_list=[]
    for epoch in range(epochs):
        data_loader=data.DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,drop_last=True)
        epoch_loss=0.0
        for batch in tqdm(data_loader,position=0):
            model.zero_grad()
            model.zero_grad()

            yhat=model.forward(batch['X'].float().to(device))

            batch_loss=F.cross_entropy(yhat,batch['y'].long().to(device))

            epoch_loss+=batch_loss.item()

            batch_loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss}')
        loss_list.append(epoch_loss)
        acc_list.append(evaluate_model)
    return acc_list,loss_list

def evaluate_model(model, dataset_val):
    with torch.no_grad():
        X=torch.from_numpy(np.array([sample['X'] for sample in dataset_val])).float()
        yhat_unnormalized=model.forward(X.to(device)).cpu().detach().numpy()
    
    yhat=np.argmax(yhat_unnormalized, axis=1)
    y=np.array([sample['y'] for sample in dataset_val])
    acc=accuracy_score(yhat,y)
    # print(yhat_unnormalized)

    print("validation acc is",acc)
    return acc



In [18]:
transform=torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    # torchvision.transforms.Normalize(mean=0,std=0.1),
    torchvision.transforms.ColorJitter(),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor()

])
train_dataset=BasicDataset(X_train,y_train,transform)
val_dataset=BasicDataset(X_val,y_val,transform)
test_dataset=BasicDataset(X_test,y_test,transform)
epochs=20
MAX_EVALS=30

In [19]:
param_grid={
    'learning_rate':list(np.logspace(np.log10(0.005),np.log10(0.5),base=10,num=1000)),
    'batch_size':[16,32,64,128,256],
    'hidden_size':[64,128,256,512,1024]
    }


best_acc=0
best_hypterparameter={}
torch.cuda.empty_cache()

for i in range(MAX_EVALS):
    hypterparameter={k:random.sample(v,1)[0] for k,v in param_grid.items()}

    lr=hypterparameter['learning_rate']
    batch_size=hypterparameter['batch_size']
    hidden_size=hypterparameter['hidden_size']

    model=RNNClassifer(hidden_size).to(device=device)
    model.train()
    train_model(model,train_dataset,lr,batch_size,epochs)
    model.eval()
    acc=evaluate_model(model,test_dataset)
    if acc>best_acc:
        best_hypterparameter=hypterparameter
        best_acc=acc
        torch.save(model.state_dict(),"best_sol.pt")
    torch.cuda.empty_cache()
print("best situation Hyperparameter :")
print(best_hypterparameter)

loss_list=[]
acc_list=[]

best_model=RNNClassifer(best_hypterparameter['hidden_size']).to(device)
best_model.load_state_dict(torch.load("best_sol.pt"))
best_model.train()
acc_list,loss_list=train_model(best_model,train_dataset,best_hypterparameter['learning_rate'],best_hypterparameter['batch_size'],200) 
best_model.eval()
acc=evaluate_model(best_model,test_dataset)
print("____________________________________")
print("The final accuracy is:",acc)
print("____________________________________")


100%|██████████| 60/60 [00:06<00:00,  9.01it/s]
Epoch 1/20 - Loss: 1586827596.0497715
100%|██████████| 60/60 [00:06<00:00,  9.09it/s]
Epoch 2/20 - Loss: 83.15593415498734
100%|██████████| 60/60 [00:06<00:00,  8.83it/s]
Epoch 3/20 - Loss: 41.65532624721527
 62%|██████▏   | 37/60 [00:04<00:02,  8.24it/s]

KeyboardInterrupt: 

In [ ]:
x_label=range(0,200)
plt.plot(x_label,loss_list,color='green')
plt.show()
plt.plot(x_label,acc_list,color='red')

In [ ]:
# best_model=torch.load("best_sol.pt")
# print(best_model)
best_model=RNNClassifer(128).to(device)
# best_model.load_state_dict(torch.load("best_sol.pt"))
train_model(best_model,train_dataset,0.007,256,20)
acc=evaluate_model(best_model,test_dataset)
print("____________________________________")
print("The final accuracy is:",acc)
print("____________________________________")



In [38]:
from sklearn.svm import SVC
svcclassfer=SVC(kernel='rbf')
X_train.tolist()
y_train.tolist()



In [34]:
print(X_train[0].shape)
def trans_dataset_gray(data):
    res=[]
    trans=torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.Grayscale(),
            torchvision.transforms.ToTensor()
        ])
    for iter in tqdm(data):
        temp=trans(torch.Tensor(iter)).numpy()
        
        res.append(np.array(temp.flatten()))
    return res
X_train_SVM=trans_dataset_gray(X_train)



100%|██████████| 7750/7750 [00:05<00:00, 1521.10it/s]


In [41]:
print(X_train_SVM[0].shape)
svcclassfer.fit(X_train_SVM,y_train)


(384,)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [42]:
X_test_SVM=trans_dataset_gray(X_test)
acc=accuracy_score(svcclassfer.predict(X_test_SVM),y_test)
print(acc)
classification_report(svcclassfer.predict(X_test_SVM),y_test)

100%|██████████| 780/780 [00:00<00:00, 2247.85it/s]
0.6333333333333333


'              precision    recall  f1-score   support\n\n           0       0.94      0.58      0.72       626\n           1       0.33      0.84      0.47       154\n\n    accuracy                           0.63       780\n   macro avg       0.63      0.71      0.60       780\nweighted avg       0.82      0.63      0.67       780\n'